<a href="https://colab.research.google.com/github/hyxxnii/Tave-6th-Project/blob/master/NIPA%20%EB%B3%B8%EC%84%A0%20-%20KB%EC%B0%A8%EC%B0%A8%EC%B0%A8%20%EC%A4%91%EA%B3%A0%EC%B0%A8%20%EB%A7%A4%EB%AC%BC%20%ED%8C%90%EB%A7%A4%EA%B8%B0%EA%B0%81%20%EC%98%88%EC%B8%A1%20%EB%AA%A8%EB%8D%B8%20%EA%B0%9C%EB%B0%9C_Try2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Load Data

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

train_path = '../data/.train/.task147/train.csv'
test_path = '../data/.train/.task147/test.csv'
submission_path = '../data/.sample_submission/.task147/sample_submission.csv'
car_df_train = pd.read_csv(train_path)

car_df_submission= pd.read_csv(submission_path)

car_df = car_df_train.copy()
car_df.head()

In [ ]:
car_df.info()

## 2. EDA

### Target 분포 확인 및 Log 변환

In [ ]:
plt.title('Originial Ad Periods Histogram')
sns.distplot(car_df['ad_periods'])

In [ ]:
# 정규분포로 변환하기 위해 로그 변환적용
plt.title('Log Transformed Ad Periods Histogram')
log_Periods = np.log1p(car_df['ad_periods'])
sns.distplot(log_Periods)

# 예측시에 다시 expm1()으로 환원

In [ ]:
# SalePrice 로그 변환
originial_Periods = car_df['ad_periods']
car_df['ad_periods'] = np.log1p(car_df['ad_periods'])

### 이상치 제거 

In [ ]:
# va피처 분포 확인
fig, axs = plt.subplots(figsize=(20,20), ncols = 6, nrows =6 )
lm_features = car_df.columns[1:35]

for i, feature in enumerate(lm_features):
    row = int(i/6)
    col = i%6
    #scatter plot
    sns.scatterplot(x=feature, y = 'ad_periods', data=car_df, ax=axs[row][col])

In [ ]:
def get_outlier(df=None, column=None, weight=1.5):
    # 1/4 분위와 3/4 분위 지점을 np.percentile로 구함
    quantile_25 = np.percentile(df[column], 25)
    quantile_75 = np.percentile(df[column], 75)
    
    # IQR을 구하고, IQR에 1.5를 곱해 최댓값과 최솟값 지점 구함
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    
    # 최댓값보다 크거나, 최솟값보다 작은 값을 이상치 데이터로 설정하고 DataFrame index 반환
    outlier_index = df[column][(df[column] < lowest_val) | (df[column] > highest_val)].index
    
    # 이상치 데이터는 중간값으로 대체
    median = np.percentile(df[column], 50)
    df[column][outlier_index] = median
    
    return outlier_index

In [ ]:
check_columns = ['va06','va07','va12','va22','va23','va25','va33']

for col in check_columns:
    outlier_index = get_outlier(df=car_df, column=col, weight=1.5)
    print('이상치 데이터 인덱스: ', outlier_index) 

#### 독립변수들 상관관계 분석

In [ ]:
onehot_list = car_df.filter(regex='^dum', axis=1).columns
onehot_list, len(onehot_list)

car_rm_dum = car_df.drop(onehot_list, axis=1, inplace=False) # 원본 데이터
car_rm_dum

In [ ]:
plt.figure(figsize=(24,20))
corr = car_rm_dum.corr()
sns.heatmap(corr, fmt='.2f', annot=True, cmap='PuBu')
plt.show()

#### category형으로 추정되는 변수 원핫 인코딩

In [ ]:
cond1 = car_df['va32'] == 5
cond2 = car_df['va24'] == 5

outlier_index = car_df[cond1 | cond2].index

print('이상치 레코드 index: ', outlier_index.values)
print('이상치 삭제 전 car_df shape: ', car_df.shape)

# DataFrame의 인덱스를 이용해 이상치 레코드 삭제
car_df = car_df.drop(outlier_index, axis=0).reset_index(drop=True)
print('이상치 삭제 후 car_df shape: ', car_df.shape)

In [ ]:
# category형으로 추정되는 변수 원핫 인코딩
# va05, va24, va32
car_df_ohe = pd.get_dummies(car_df, columns=['va05','va24','va32'])
car_df_ohe.head()

#### 연속형 변수들만 skew(왜도) 1이상 로그 변환

In [ ]:
# 피처 데이터 세트의 데이터 분포도 확인
from scipy.stats import skew

col = ['va01','va02','va03','va04','va06','va12','va13','va22','va23','va25','va27','va28','va33']
skew_features = car_df_ohe[col].apply(lambda x:skew(x))

# skew(왜곡) 정도가 1 이상인 칼럼만 추출
skew_features_top = skew_features[skew_features > 1]
print(skew_features_top.sort_values(ascending=False))

In [ ]:
# 추출된 왜곡 정도가 높은 피처를 로그 변환
car_df_ohe[skew_features_top.index] = np.log1p(car_df_ohe[skew_features_top.index])

# Base Model
## - Linear Regression, Ridge, Lasso, RandomForest, XGBoost

In [ ]:
from sklearn.metrics import mean_squared_error

def get_rmse(model):
    pred = np.expm1(model.predict(X_test))
    mse = mean_squared_error(y_test_exp, pred)
    rmse = np.sqrt(mse)
    print(model.__class__.__name__, 'RMSE: ', np.round(rmse,3))
    return rmse

def get_rmses(models):
    rmses = []
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return np.round(rmses, 3)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

y_target = car_df_ohe['ad_periods'] # 로그 변환됨
X_features = car_df_ohe.drop('ad_periods', axis=1, inplace=False) # inplace=False 주의
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=156)

# LinearRegression, Ridge, Lasso 학습/예측/평가
lr_reg = LinearRegression()
ridge_reg = Ridge()
lasso_reg = Lasso()
rf_reg = RandomForestRegressor(n_estimators=500, max_depth=6, random_state=156)
xgb_reg = XGBRegressor(n_estimators=500, max_depth=6, random_state=156)
lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.01, num_leaves=4, random_state=156,
                         subsample=0.8, colsample_bytree=0.4)
  
lr_reg.fit(X_train, y_train)
ridge_reg.fit(X_train, y_train)
lasso_reg.fit(X_train, y_train)
rf_reg.fit(X_train, y_train)

lr_pred = lr_reg.predict(X_test)
ridge_pred = ridge_reg.predict(X_test)
lasso_pred = lasso_reg.predict(X_test)
rf_pred = rf_reg.predict(X_test)

y_test_exp = np.expm1(y_test)

models = [lr_reg, ridge_reg, lasso_reg, rf_reg]
get_rmses(models)

In [ ]:
lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.01, num_leaves=4, random_state=156,
                         subsample=0.8, colsample_bytree=0.4)

lgbm_reg.fit(X_train, y_train, eval_metric='rmse')
get_rmse(lgbm_reg)

In [ ]:
xgb_reg.fit(X_train, y_train, early_stopping_rounds=50,
           eval_metric='rmse', eval_set=[(X_train, y_train), (X_test, y_test)])

get_rmse(xgb_reg)

In [ ]:
# 모델 저장
import joblib

joblib.dump(xgb_reg, './xgb_model_yh.pkl')